In [90]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager  # Import webdriver_manager
import time

In [92]:
# Initialize Chrome WebDriver using webdriver_manager to automatically handle the ChromeDriver download
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")  # Open window maximized

# Use webdriver_manager
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    driver.get("https://www.weather.gov/lox/observations_historical")

    # Click on Ventura
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.LINK_TEXT, "Ventura"))).click()

    # Select KOXR
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.LINK_TEXT, "KOXR*"))).click()

    # Select start year and end year
    select_syear = Select(WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "syear"))))
    select_eyear = Select(WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "eyear"))))

    # Set start year to 2000 and end year to 2024
    select_syear.select_by_visible_text("2000")
    select_eyear.select_by_visible_text("2024")

    # Select checkboxes for variables
    checkboxes = [1, 6, 22, 23, 26, 27, 28]  # Adjust indices if needed
    for checkbox in checkboxes:
        checkbox_xpath = f"//div[@id='launchpad']/span[{checkbox}]/label"
        WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, checkbox_xpath))).click()

    # Clicking the submit button to process the request
    submit_button_xpath = "//button[@onclick=\"javascript:routeStudy('1','KOXR','grabObs')\"]"
    WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, submit_button_xpath))).click()
    
    # Downloading the file
    download_xpath = "//a[contains(text(),'Download')]"
    WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, download_xpath))).click()

    print("Downloaded data for 2000 to 2024")
    
    # Click on the "Go Again" button to return to Ventura and KOXR
    go_again_button_xpath = "//a[text()='Go Again']"
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, go_again_button_xpath))).click()

    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.LINK_TEXT, "Ventura"))).click()
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.LINK_TEXT, "KOXR*"))).click()

    time.sleep(10)
    
except Exception as e:
    print(f"Error occurred: {e}")
finally:
    driver.quit()


Error occurred: Message: 
Stacktrace:
0   chromedriver                        0x0000000104ed36c8 cxxbridge1$str$ptr + 2791212
1   chromedriver                        0x0000000104ecbc9c cxxbridge1$str$ptr + 2759936
2   chromedriver                        0x0000000104a1de30 cxxbridge1$string$len + 92928
3   chromedriver                        0x0000000104a65170 cxxbridge1$string$len + 384576
4   chromedriver                        0x0000000104aa65f8 cxxbridge1$string$len + 651976
5   chromedriver                        0x0000000104a592fc cxxbridge1$string$len + 335820
6   chromedriver                        0x0000000104e986c4 cxxbridge1$str$ptr + 2549544
7   chromedriver                        0x0000000104e9b988 cxxbridge1$str$ptr + 2562540
8   chromedriver                        0x0000000104e7871c cxxbridge1$str$ptr + 2418560
9   chromedriver                        0x0000000104e9c1e8 cxxbridge1$str$ptr + 2564684
10  chromedriver                        0x0000000104e69750 cxxbridge1$str$p

In [94]:
import pandas as pd

file_path = 'KOXR_2000-2024.csv'

df = pd.read_csv(file_path)

# Parse the 'date_time' column
df['date_time'] = pd.to_datetime(df['date_time'], format='%m/%d/%y-%H:%M')

df_sorted = df.sort_values(by='date_time')

# Reset the index
df_sorted = df_sorted.reset_index(drop=True)

# Display the first few rows of the sorted DataFrame
print(df_sorted.head())

# Specify the output path
output_path = 'KOXR_2000-2024_sorted.csv'

# Export the sorted DataFrame to a new CSV file
df_sorted.to_csv(output_path, index=False)

print(f"File has been saved to {output_path}")

/var/folders/mw/5td269yx439_x7m8tft2f1g80000gn/T/ipykernel_14933/4170102582.py:5: DtypeWarning: Columns (3,4,5,6,7,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


       Date   Time           date_time altimeter air_temp relative_humidity  \
0  12/31/99  16:50 1999-12-31 16:50:00     30.02     54.0              66.3   
1  12/31/99  17:50 1999-12-31 17:50:00     30.03     52.0              71.3   
2  12/31/99  18:50 1999-12-31 18:50:00     30.04     52.0              74.3   
3  12/31/99  19:15 1999-12-31 19:15:00     30.04     51.8              71.3   
4  12/31/99  19:50 1999-12-31 19:50:00     30.04     52.0              71.3   

  wind_speed wind_direction sea_level_pressure wind_gust Unnamed: 10  
0      10.36          280.0             1016.7       NaN     1016.61  
1       6.91          270.0             1016.8       NaN     1016.95  
2       6.91          270.0             1017.0       NaN     1017.29  
3       6.91          270.0                NaN       NaN     1017.29  
4       3.44          250.0             1017.2       NaN     1017.29  
File has been saved to KOXR_2000-2024_sorted.csv


In [96]:
# Drop the 'date_time' column and 'wind_gust' from the sorted DataFrame
df_sorted = df_sorted.drop(columns=['date_time', 'wind_gust'])

# Rename the 'Unnamed: 10' column to 'Atmospheric Pressure'
df_sorted = df_sorted.rename(columns={'Unnamed: 10': 'Atmospheric Pressure'})

# Handling Date and Time
df_sorted['Date'] = pd.to_datetime(df_sorted['Date'], format='%m/%d/%y', errors='coerce').dt.date

df_sorted['Time'] = pd.to_datetime(df_sorted['Time'], format='%H:%M', errors='coerce').dt.time

# Select columns to convert to numeric
columns_to_convert = [col for col in df_sorted.columns if col not in ['Date', 'Time']]

# Convert all columns except 'Date' and 'Time' to numeric
for column in columns_to_convert:
    df_sorted[column] = pd.to_numeric(df_sorted[column], errors='coerce')

In [98]:
print(df_sorted.head(5))

         Date      Time  altimeter  air_temp  relative_humidity  wind_speed  \
0  1999-12-31  16:50:00      30.02      54.0               66.3       10.36   
1  1999-12-31  17:50:00      30.03      52.0               71.3        6.91   
2  1999-12-31  18:50:00      30.04      52.0               74.3        6.91   
3  1999-12-31  19:15:00      30.04      51.8               71.3        6.91   
4  1999-12-31  19:50:00      30.04      52.0               71.3        3.44   

   wind_direction  sea_level_pressure  Atmospheric Pressure  
0           280.0              1016.7               1016.61  
1           270.0              1016.8               1016.95  
2           270.0              1017.0               1017.29  
3           270.0                 NaN               1017.29  
4           250.0              1017.2               1017.29  


In [100]:
print(df_sorted.dtypes)

Date                     object
Time                     object
altimeter               float64
air_temp                float64
relative_humidity       float64
wind_speed              float64
wind_direction          float64
sea_level_pressure      float64
Atmospheric Pressure    float64
dtype: object


In [102]:
# This would ensure that the date and time columns are seperate but be
# able to be used for a dashboard since it is not in a string format anymore.
print(type(df_sorted.loc[0, 'Date']))
print(type(df_sorted.loc[0, 'Time']))

<class 'datetime.date'>
<class 'datetime.time'>


In [104]:
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect('KOXR_2000-2024_data.db')  # This creates the database file.

# Write the DataFrame to an SQLite table
df_sorted.to_sql('weather', conn, if_exists='replace', index=False)

# Commit and close the connection
conn.commit()
conn.close()

print("Data has been successfully inserted into the database.")

Data has been successfully inserted into the database.
